In [1]:
import os
import time
import certifi
import requests
import pandas as pd
import xml.etree.ElementTree as ET

# Usa el bundle de certifi para evitar errores de certificado en requests
os.environ.setdefault("REQUESTS_CA_BUNDLE", certifi.where())
os.environ.setdefault("SSL_CERT_FILE", certifi.where())


'/home/pablo/.local/lib/python3.10/site-packages/certifi/cacert.pem'

In [2]:
base_url = catalog.load('params:oai_fetch_options.base_url')
context = catalog.load('params:oai_fetch_options.context')
env = 'dev'


                    INFO     Loading data from params:oai_fetch_options.base_url                ]8;id=442731;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=514519;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from params:oai_fetch_options.context                 ]8;id=918318;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=731703;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

In [3]:
VERIFY_SSL = os.getenv("OAI_VERIFY_SSL", "false").lower() == "true"
CA_BUNDLE = os.getenv("OAI_CA_BUNDLE") or certifi.where()

# Pon VERIFY_SSL=True si quieres validar el certificado con CA_BUNDLE; se desactiva por defecto


In [4]:
base_url

'https://ri.conicet.gov.ar/oai/'

La función **get_oai_records()** también se usa para recuperar items en otro nodo

In [5]:
def get_oai_records(base_url, verify=None):
    start_time = time.time()

    verify_param = CA_BUNDLE if VERIFY_SSL else False
    if verify is not None:
        verify_param = verify

    response = requests.get(base_url, verify=verify_param)
    end_time = time.time()
    elapsed_time = end_time - start_time

    print(f"Sleeping for {elapsed_time:.2f} seconds")
    time.sleep(elapsed_time)

    if response.status_code == 200:
        return response
    else:
        print(f"Error: {response.status_code}")
        return None


In [6]:
def oai_extract_sets(base_url, context, env, verify=None):

    resumption_token = 0
    all_sets = []

    iteration_limit = 2
    iteration_count = 0

    while True:

        if env == 'dev' and iteration_count >= iteration_limit:
            break

        params = f'/{context}?verb=ListSets&resumptionToken=////{resumption_token}'
        url = base_url + params

        print(f"Consultando: {url}")

        response = get_oai_records(url, verify=verify)
        if not response:
            break

        xml_content = response.text
        root = ET.fromstring(xml_content)
        ns = {'oai': 'http://www.openarchives.org/OAI/2.0/'}

        sets_data = []
        for set_elem in root.findall('.//oai:set', ns):
            set_spec = set_elem.find('oai:setSpec', ns).text if set_elem.find('oai:setSpec', ns) is not None else None
            set_name = set_elem.find('oai:setName', ns).text if set_elem.find('oai:setName', ns) is not None else None
            sets_data.append({'setSpec': set_spec, 'setName': set_name})

        if not sets_data:
            print("No se encontraron más sets.")
            break

        all_sets.extend(sets_data)
        resumption_token += 100  # avanzar manualmente
        iteration_count += 1

    df_sets = pd.DataFrame(all_sets)
    return df_sets


In [7]:
df = oai_extract_sets(base_url, context, env)


Consultando: https://ri.conicet.gov.ar/oai//request?verb=ListSets&resumptionToken=////0


                    WARNING  /home/pablo/.local/lib/python3.10/site-packages/urllib3/connectionpool ]8;id=193430;file:///usr/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=521922;file:///usr/lib/python3.10/warnings.py#109\109]8;;\
                             .py:1099: InsecureRequestWarning: Unverified HTTPS request is being                   
                             made to host 'ri.conicet.gov.ar'. Adding certificate verification is                  
                             strongly advised. See:                                                                
                             https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warni                
                             ngs                                                                                   
                               warnings.warn(                                                                      
                                                                                                                   

Sleeping for 2.88 seconds
Consultando: https://ri.conicet.gov.ar/oai//request?verb=ListSets&resumptionToken=////100


[11/21/25 20:33:06] WARNING  /home/pablo/.local/lib/python3.10/site-packages/urllib3/connectionpool ]8;id=240615;file:///usr/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=325770;file:///usr/lib/python3.10/warnings.py#109\109]8;;\
                             .py:1099: InsecureRequestWarning: Unverified HTTPS request is being                   
                             made to host 'ri.conicet.gov.ar'. Adding certificate verification is                  
                             strongly advised. See:                                                                
                             https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warni                
                             ngs                                                                                   
                               warnings.warn(                                                                      
                                                                                                                   

Sleeping for 0.39 seconds


In [8]:
df = oai_extract_sets(base_url, context, env, verify=CA_BUNDLE if VERIFY_SSL else False)


Consultando: https://ri.conicet.gov.ar/oai//request?verb=ListSets&resumptionToken=////0


[11/21/25 20:33:07] WARNING  /home/pablo/.local/lib/python3.10/site-packages/urllib3/connectionpool ]8;id=642582;file:///usr/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=435259;file:///usr/lib/python3.10/warnings.py#109\109]8;;\
                             .py:1099: InsecureRequestWarning: Unverified HTTPS request is being                   
                             made to host 'ri.conicet.gov.ar'. Adding certificate verification is                  
                             strongly advised. See:                                                                
                             https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warni                
                             ngs                                                                                   
                               warnings.warn(                                                                      
                                                                                                                   

Sleeping for 0.19 seconds
Consultando: https://ri.conicet.gov.ar/oai//request?verb=ListSets&resumptionToken=////100


                    WARNING  /home/pablo/.local/lib/python3.10/site-packages/urllib3/connectionpool ]8;id=632867;file:///usr/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=775716;file:///usr/lib/python3.10/warnings.py#109\109]8;;\
                             .py:1099: InsecureRequestWarning: Unverified HTTPS request is being                   
                             made to host 'ri.conicet.gov.ar'. Adding certificate verification is                  
                             strongly advised. See:                                                                
                             https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warni                
                             ngs                                                                                   
                               warnings.warn(                                                                      
                                                                                                                   

Sleeping for 0.18 seconds


In [9]:
df

,setSpec,setName
0,snrd,Sistema Nacional de Repositorios Digitales
1,com_11336_73,AREA DE INFLUENCIA CENTRO CIENTÍFICO TECNOLÓGI...
2,com_11336_116,AREA DE INFLUENCIA CENTRO CIENTÍFICO TECNOLÓGI...
3,com_11336_169,AREA DE INFLUENCIA CENTRO CIENTÍFICO TECNOLÓGI...
4,com_11336_184,AREA DE INFLUENCIA CENTRO CIENTÍFICO TECNOLÓGI...
...,...,...
195,com_11336_174,IIB - INSTITUTO DE INVESTIGACIONES BIOLÓGICAS
196,com_11336_167,IIB-INTECH - INSTITUTO DE INVESTIGACIONES BIOT...
197,com_11336_108933,IIBIO - INSTITUTO DE INVESTIGACIONES BIOTECNOL...
198,com_11336_104,IIBYT - INSTITUTO DE INVESTIGACIONES BIOLÓGICA...
